In [0]:
spark.conf.set("CONFIG HERE")

# Read Data From Bronze Layer

### Customers data

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
df_customer = spark.read.csv("abfss://bronze@dlpracticenajeeb.dfs.core.windows.net/SalesLT/Customer.csv", header=True, inferSchema=True)
df_customer.limit(5).display()

CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
1,false,Mr.,Orlando,N.,Gee,null,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,3f5ae95e-b87d-4aed-95b4-c3797afcb74f,2005-08-01T00:00:00Z
2,false,Mr.,Keith,null,Harris,null,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,e552f657-a9af-4a7d-a645-c429d6e02491,2006-08-01T00:00:00Z
3,false,Ms.,Donna,F.,Carreras,null,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2005-09-01T00:00:00Z
4,false,Ms.,Janet,M.,Gates,null,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,ff862851-1daa-4044-be7c-3e85583c054d,2006-07-01T00:00:00Z
5,false,Mr.,Lucy,null,Harrington,null,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,83905bdc-6f5e-4f71-b162-c98da069f38a,2006-09-01T00:00:00Z


#### Drop Columns

In [0]:
df_customer = df_customer.drop('PasswordHash', 'PasswordSalt', 'NameStyle', 'Suffix', 'rowguid')
df_customer.limit(5).display()

CustomerID,Title,FirstName,MiddleName,LastName,CompanyName,SalesPerson,EmailAddress,Phone,ModifiedDate
1,Mr.,Orlando,N.,Gee,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,2005-08-01T00:00:00Z
2,Mr.,Keith,null,Harris,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,2006-08-01T00:00:00Z
3,Ms.,Donna,F.,Carreras,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,2005-09-01T00:00:00Z
4,Ms.,Janet,M.,Gates,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,2006-07-01T00:00:00Z
5,Mr.,Lucy,null,Harrington,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,2006-09-01T00:00:00Z


#### Replace Null value in Middles name

In [0]:
df_customer = df_customer.fillna('No Middle Name', subset=['MiddleName'])
df_customer.limit(5).display()

CustomerID,Title,FirstName,MiddleName,LastName,CompanyName,SalesPerson,EmailAddress,Phone,ModifiedDate
1,Mr.,Orlando,N.,Gee,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,2005-08-01T00:00:00Z
2,Mr.,Keith,No Middle Name,Harris,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,2006-08-01T00:00:00Z
3,Ms.,Donna,F.,Carreras,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,2005-09-01T00:00:00Z
4,Ms.,Janet,M.,Gates,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,2006-07-01T00:00:00Z
5,Mr.,Lucy,No Middle Name,Harrington,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,2006-09-01T00:00:00Z


#### Split the Sales person column with \ delimiter to get the sales person name


In [0]:
df_customer = df_customer.withColumn('SalesPerson', split('SalesPerson', '\\\\')[1])
df_customer.limit(5).display()

CustomerID,Title,FirstName,MiddleName,LastName,CompanyName,SalesPerson,EmailAddress,Phone,ModifiedDate
1,Mr.,Orlando,N.,Gee,A Bike Store,pamela0,orlando0@adventure-works.com,245-555-0173,2005-08-01T00:00:00Z
2,Mr.,Keith,No Middle Name,Harris,Progressive Sports,david8,keith0@adventure-works.com,170-555-0127,2006-08-01T00:00:00Z
3,Ms.,Donna,F.,Carreras,Advanced Bike Components,jillian0,donna0@adventure-works.com,279-555-0130,2005-09-01T00:00:00Z
4,Ms.,Janet,M.,Gates,Modular Cycle Systems,jillian0,janet1@adventure-works.com,710-555-0173,2006-07-01T00:00:00Z
5,Mr.,Lucy,No Middle Name,Harrington,Metropolitan Sports Supply,shu0,lucy0@adventure-works.com,828-555-0186,2006-09-01T00:00:00Z


#### Change Date Format

In [0]:
df_customer = df_customer.withColumn('ModifiedDate', date_format('ModifiedDate', 'dd/MM/yyyy'))
df_customer.limit(5).display()

CustomerID,Title,FirstName,MiddleName,LastName,CompanyName,SalesPerson,EmailAddress,Phone,ModifiedDate
1,Mr.,Orlando,N.,Gee,A Bike Store,pamela0,orlando0@adventure-works.com,245-555-0173,01/08/2005
2,Mr.,Keith,No Middle Name,Harris,Progressive Sports,david8,keith0@adventure-works.com,170-555-0127,01/08/2006
3,Ms.,Donna,F.,Carreras,Advanced Bike Components,jillian0,donna0@adventure-works.com,279-555-0130,01/09/2005
4,Ms.,Janet,M.,Gates,Modular Cycle Systems,jillian0,janet1@adventure-works.com,710-555-0173,01/07/2006
5,Mr.,Lucy,No Middle Name,Harrington,Metropolitan Sports Supply,shu0,lucy0@adventure-works.com,828-555-0186,01/09/2006


#### Check if there is any Nulls in data

In [0]:
df_customer.select([
    sum(col(c).isNull().cast('int')).alias(c) for c in df_customer.columns
]).display()

CustomerID,Title,FirstName,MiddleName,LastName,CompanyName,SalesPerson,EmailAddress,Phone,ModifiedDate
0,7,0,0,0,0,0,0,0,0


#### Check how many time a Title is repeated

In [0]:
df_customer.groupBy('Title').count().orderBy('count', ascending=False).display()

Title,count
Mr.,490
Ms.,340
null,7
Sr.,6
Sra.,4


#### Create a Gender Column by Using title Column

In [0]:
df_customer = df_customer.withColumn('Gender', when(col('Title').isin(['Ms.', 'Sra.']), 'Female')\
    .when(col('Title').isNull(), 'Undefined')\
    .otherwise('Male'))

In [0]:
df_customer.limit(5).display()

CustomerID,Title,FirstName,MiddleName,LastName,CompanyName,SalesPerson,EmailAddress,Phone,ModifiedDate,Gender
1,Mr.,Orlando,N.,Gee,A Bike Store,pamela0,orlando0@adventure-works.com,245-555-0173,01/08/2005,Male
2,Mr.,Keith,No Middle Name,Harris,Progressive Sports,david8,keith0@adventure-works.com,170-555-0127,01/08/2006,Male
3,Ms.,Donna,F.,Carreras,Advanced Bike Components,jillian0,donna0@adventure-works.com,279-555-0130,01/09/2005,Female
4,Ms.,Janet,M.,Gates,Modular Cycle Systems,jillian0,janet1@adventure-works.com,710-555-0173,01/07/2006,Female
5,Mr.,Lucy,No Middle Name,Harrington,Metropolitan Sports Supply,shu0,lucy0@adventure-works.com,828-555-0186,01/09/2006,Male


#### Save Transformed File to Silver Layer

In [0]:
df_customer.write.format('delta')\
    .mode('overwrite')\
        .save('abfss://silver@dlpracticenajeeb.dfs.core.windows.net/Customers')

### Address Data

In [0]:
df_address = spark.read.format('csv').\
    option('header', True).\
    option('inferSchema', True).\
    load('abfss://bronze@dlpracticenajeeb.dfs.core.windows.net/SalesLT/Address.csv')
df_address.limit(5).display()

AddressID,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,rowguid,ModifiedDate
9,8713 Yosemite Ct.,null,Bothell,Washington,United States,98011,268af621-76d7-4c78-9441-144fd139821a,2006-07-01T00:00:00Z
11,1318 Lasalle Street,null,Bothell,Washington,United States,98011,981b3303-aca2-49c7-9a96-fb670785b269,2007-04-01T00:00:00Z
25,9178 Jumping St.,null,Dallas,Texas,United States,75201,c8df3bd9-48f0-4654-a8dd-14a67a84d3c6,2006-09-01T00:00:00Z
28,9228 Via Del Sol,null,Phoenix,Arizona,United States,85004,12ae5ee1-fc3e-468b-9b92-3b970b169774,2005-09-01T00:00:00Z
32,26910 Indela Road,null,Montreal,Quebec,Canada,H1Y 2H5,84a95f62-3ae8-4e7e-bbd5-5a6f00cd982d,2006-08-01T00:00:00Z


In [0]:
df_address = df_address.drop('AddressLine1', 'AddressLine2', 'rowguid', 'PostalCode')
df_address.limit(5).display()

AddressID,City,StateProvince,CountryRegion,ModifiedDate
9,Bothell,Washington,United States,2006-07-01T00:00:00Z
11,Bothell,Washington,United States,2007-04-01T00:00:00Z
25,Dallas,Texas,United States,2006-09-01T00:00:00Z
28,Phoenix,Arizona,United States,2005-09-01T00:00:00Z
32,Montreal,Quebec,Canada,2006-08-01T00:00:00Z


#### Change Date Format

In [0]:
df_address = df_address.withColumn('ModifiedDate', date_format('ModifiedDate', 'dd/MM/yyyy'))
df_address.limit(5).display()

AddressID,City,StateProvince,CountryRegion,ModifiedDate
9,Bothell,Washington,United States,01/07/2006
11,Bothell,Washington,United States,01/04/2007
25,Dallas,Texas,United States,01/09/2006
28,Phoenix,Arizona,United States,01/09/2005
32,Montreal,Quebec,Canada,01/08/2006


#### Check Null Values in all columns

In [0]:
df_address.select(
    [sum(col(c).isNull().cast('int')).alias(c) for c in df_address.columns]
).display()

AddressID,City,StateProvince,CountryRegion,ModifiedDate
0,0,0,0,0


#### Write file to silver container

In [0]:
df_address.write.format('delta').mode('overwrite').\
    option('path', 'abfss://silver@dlpracticenajeeb.dfs.core.windows.net/Address').\
        save()

### Customer Address File

In [0]:
df_cus_ad = spark.read.format('csv').option('header',  True).\
    option('inferSchema', True).\
        load('abfss://bronze@dlpracticenajeeb.dfs.core.windows.net/SalesLT/CustomerAddress.csv')
df_cus_ad.limit(5).display()

CustomerID,AddressID,AddressType,rowguid,ModifiedDate
29485,1086,Main Office,16765338-dbe4-4421-b5e9-3836b9278e63,2007-09-01T00:00:00Z
29486,621,Main Office,22b3e910-14af-4ed5-8b4d-23bbe757414d,2005-09-01T00:00:00Z
29489,1069,Main Office,a095c88b-d7e6-4178-a078-2eca44214801,2005-07-01T00:00:00Z
29490,887,Main Office,f12e1702-d897-4035-b614-0fe2c72168a9,2006-09-01T00:00:00Z
29492,618,Main Office,5b3b3eb2-3f43-47ed-a20c-23697dabf23b,2006-12-01T00:00:00Z


#### Transform Data

In [0]:
df_cus_ad = df_cus_ad.drop('rowguid')

In [0]:
df_cus_ad = df_cus_ad.withColumn('modifiedDate', date_format('ModifiedDate', 'dd/MM/yyyy'))

#### Check Null Values

In [0]:
df_cus_ad.select([
    sum(col(c).isNull().cast('int')).alias(c) for c in df_cus_ad.columns
]).display()

CustomerID,AddressID,AddressType,modifiedDate
0,0,0,0


#### Write  to Silver Container

In [0]:
df_cus_ad.write.format('delta').mode('overwrite').option('path', 'abfss://silver@dlpracticenajeeb.dfs.core.windows.net/CustomerAddress').save()

### Product File

In [0]:
df_prod = spark.read.format('csv').option('header', True).\
    option('inferSchema', True).\
        load('abfss://bronze@dlpracticenajeeb.dfs.core.windows.net/SalesLT/Product.csv')
df_prod.limit(5).display()

ProductID,Name,ProductNumber,Color,StandardCost,ListPrice,Size,Weight,ProductCategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,ThumbNailPhoto,ThumbnailPhotoFileName,rowguid,ModifiedDate
680,"HL Road Frame - Black, 58",FR-R92B-58,Black,1059.31,1431.5,58,1016.04,18,6,2002-06-01T00:00:00Z,null,null,0x47494638396150003100F70000000000800000008000808000000080800080008080808080C0C0C0FF000000FF00FFFF000000FFFF00FF00FFFFFFFFFF0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000330000660000990000CC0000FF0033000033330033660033990033CC0033FF0066000066330066660066990066CC0066FF0099000099330099660099990099CC0099FF00CC0000CC3300CC6600CC9900CCCC00CCFF00FF0000FF3300FF6600FF9900FFCC00FFFF3300003300333300663300993300CC3300FF3333003333333333663333993333CC3333FF3366003366333366663366993366CC3366FF3399003399333399663399993399CC3399FF33CC0033CC3333CC6633CC9933CCCC33CCFF33FF0033FF3333FF6633FF9933FFCC33FFFF6600006600336600666600996600CC6600FF6633006633336633666633996633CC6633FF6666006666336666666666996666CC6666FF6699006699336699666699996699CC6699FF66CC0066CC3366CC6666CC9966CCCC66CCFF66FF0066FF3366FF6666FF9966FFCC66FFFF9900009900339900669900999900CC9900FF9933009933339933669933999933CC9933FF9966009966339966669966999966CC9966FF9999009999339999669999999999CC9999FF99CC0099CC3399CC6699CC9999CCCC99CCFF99FF0099FF3399FF6699FF9999FFCC99FFFFCC0000CC0033CC0066CC0099CC00CCCC00FFCC3300CC3333CC3366CC3399CC33CCCC33FFCC6600CC6633CC6666CC6699CC66CCCC66FFCC9900CC9933CC9966CC9999CC99CCCC99FFCCCC00CCCC33CCCC66CCCC99CCCCCCCCCCFFCCFF00CCFF33CCFF66CCFF99CCFFCCCCFFFFFF0000FF0033FF0066FF0099FF00CCFF00FFFF3300FF3333FF3366FF3399FF33CCFF33FFFF6600FF6633FF6666FF6699FF66CCFF66FFFF9900FF9933FF9966FF9999FF99CCFF99FFFFCC00FFCC33FFCC66FFCC99FFCCCCFFCCFFFFFF00FFFF33FFFF66FFFF99FFFFCCFFFFFF21F90401000010002C00000000500031000008FF00FF091C48B0A0C18308132A5CC8B0A1C38710234A9C48B1A2C58B18336ADCC8B1A3C78F20438A1C49B2A4C9932853AA5C9911058A812E17C664F9D0E5CB7F3313E6A4C9D0A6C099366FC27C19D367CEA04371DE44CA1169D1A542952A25FA1467D2A7547F46C558542AD0A855A58A9D9AD52BD4A654C3EE547BD6AC58AC3E377E2DBB96EED9AF6FCB7A9C9B97205BAD7DB1F6952B74EED6BF79E3B2DDC95362E1AD8D2F328D4CB9B2E5CB9831C77518D62063C6223743040DD82F6490558FA6BD0B152F5EB2A8B3AA7EEC96B651D96D3BFE9D8D7BE86EBB1F7F034ECD3AF06AD292831237EEB6F9F290AAB5B65DEE9A7A6EC249A7FA7EAEDC79EBD399C38B121F4FBEBCF9F3E8D3AB5FCFBEBDFBF7EF0302003B,no_image_available_small.gif,43dd68d6-14a4-461f-9069-55309d90ea7e,2008-03-11T10:01:36.827Z
706,"HL Road Frame - Red, 58",FR-R92R-58,Red,1059.31,1431.5,58,1016.04,18,6,2002-06-01T00:00:00Z,null,null,0x47494638396150003100F70000000000800000008000808000000080800080008080808080C0C0C0FF000000FF00FFFF000000FFFF00FF00FFFFFFFFFF0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000330000660000990000CC0000FF0033000033330033660033990033CC0033FF0066000066330066660066990066CC0066FF0099000099330099660099990099CC0099FF00CC0000CC3300CC6600CC9900CCCC00CCFF00FF0000FF3300FF6600FF9900FFCC00FFFF3300003300333300663300993300CC3300FF3333003333333333663333993333CC3333FF3366003366333366663366993366CC3366FF3399003399333399663399993399CC3399FF33CC0033CC3333CC6633CC9933CCCC33CCFF33FF0033FF3333FF6633FF9933FFCC33FFFF6600006600336600666600996600CC6600FF6633006633336633666633996633CC6633FF6666006666336666666666996666CC6666FF6699006699336699666699996699CC6699FF66CC0066CC3366CC6666CC9966CCCC66CCFF66FF0066FF3366FF6666FF9966FFCC66FFFF9900009900339900669900999900CC9900FF9933009933339933669933999933CC9933FF9966009966339966669966999966CC9966FF9999009999339999669999999999CC9999FF99CC0099CC3399CC6699CC9999CCCC99CCFF99FF0099FF3399FF6699FF9999FFCC99FFFFCC0000CC0033CC0066CC0099CC00CCCC00FFCC3300CC3333CC3366CC3399CC33CCCC33FFCC6600CC6633CC6666CC6699CC66CCCC66FFCC9900CC9933CC9966CC9999CC99CCCC99FFCCCC00CCCC33CCCC66CCCC99CCCCCCCCCCFFCCFF00CCFF33CCFF6

#### Drop Columns

In [0]:
df_prod = df_prod.drop('rowguid', 'ThumbNailPhoto', 'ThumbnailPhotoFileName', 'DiscontinuedDate')
df_prod.limit(5).display()

ProductID,Name,ProductNumber,Color,StandardCost,ListPrice,Size,Weight,ProductCategoryID,ProductModelID,SellStartDate,SellEndDate,ModifiedDate
680,"HL Road Frame - Black, 58",FR-R92B-58,Black,1059.31,1431.5,58,1016.04,18,6,2002-06-01T00:00:00Z,null,2008-03-11T10:01:36.827Z
706,"HL Road Frame - Red, 58",FR-R92R-58,Red,1059.31,1431.5,58,1016.04,18,6,2002-06-01T00:00:00Z,null,2008-03-11T10:01:36.827Z
707,"Sport-100 Helmet, Red",HL-U509-R,Red,13.0863,34.99,null,null,35,33,2005-07-01T00:00:00Z,null,2008-03-11T10:01:36.827Z
708,"Sport-100 Helmet, Black",HL-U509,Black,13.0863,34.99,null,null,35,33,2005-07-01T00:00:00Z,null,2008-03-11T10:01:36.827Z
709,"Mountain Bike Socks, M",SO-B909-M,White,3.3963,9.5,M,null,27,18,2005-07-01T00:00:00Z,2006-06-30T00:00:00Z,2008-03-11T10:01:36.827Z


#### Fill Null in SellEndDate with today's Date

In [0]:
df_prod = df_prod.withColumn('SellEndDate', when(col('SellEndDate').isNull(), current_date()).otherwise(col('SellEndDate')))
df_prod.limit(5).display()

ProductID,Name,ProductNumber,Color,StandardCost,ListPrice,Size,Weight,ProductCategoryID,ProductModelID,SellStartDate,SellEndDate,ModifiedDate
680,"HL Road Frame - Black, 58",FR-R92B-58,Black,1059.31,1431.5,58,1016.04,18,6,2002-06-01T00:00:00Z,2025-04-08T00:00:00Z,2008-03-11T10:01:36.827Z
706,"HL Road Frame - Red, 58",FR-R92R-58,Red,1059.31,1431.5,58,1016.04,18,6,2002-06-01T00:00:00Z,2025-04-08T00:00:00Z,2008-03-11T10:01:36.827Z
707,"Sport-100 Helmet, Red",HL-U509-R,Red,13.0863,34.99,null,null,35,33,2005-07-01T00:00:00Z,2025-04-08T00:00:00Z,2008-03-11T10:01:36.827Z
708,"Sport-100 Helmet, Black",HL-U509,Black,13.0863,34.99,null,null,35,33,2005-07-01T00:00:00Z,2025-04-08T00:00:00Z,2008-03-11T10:01:36.827Z
709,"Mountain Bike Socks, M",SO-B909-M,White,3.3963,9.5,M,null,27,18,2005-07-01T00:00:00Z,2006-06-30T00:00:00Z,2008-03-11T10:01:36.827Z


#### Create Total Sale Duration Column (SellStartDate-SellEndDate)

In [0]:
df_prod = df_prod.withColumn('SellDuration', datediff('SellEndDate', 'SellStartDate'))

#### Format dates

In [0]:
df_prod = df_prod.withColumn('modifiedDate', date_format('ModifiedDate', 'dd/MM/yyyy')).\
                withColumn('SellStartDate', date_format('SellStartDate', 'dd/MM/yyyy')).\
                    withColumn('SellEndDate', date_format('SellEndDate', 'dd/MM/yyyy'))
df_prod.limit(5).display()

ProductID,Name,ProductNumber,Color,StandardCost,ListPrice,Size,Weight,ProductCategoryID,ProductModelID,SellStartDate,SellEndDate,modifiedDate,SellDuration
680,"HL Road Frame - Black, 58",FR-R92B-58,Black,1059.31,1431.5,58,1016.04,18,6,01/06/2002,08/04/2025,11/03/2008,8347
706,"HL Road Frame - Red, 58",FR-R92R-58,Red,1059.31,1431.5,58,1016.04,18,6,01/06/2002,08/04/2025,11/03/2008,8347
707,"Sport-100 Helmet, Red",HL-U509-R,Red,13.0863,34.99,null,null,35,33,01/07/2005,08/04/2025,11/03/2008,7221
708,"Sport-100 Helmet, Black",HL-U509,Black,13.0863,34.99,null,null,35,33,01/07/2005,08/04/2025,11/03/2008,7221
709,"Mountain Bike Socks, M",SO-B909-M,White,3.3963,9.5,M,null,27,18,01/07/2005,30/06/2006,11/03/2008,364


#### Print Schema


In [0]:
df_prod.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- ProductNumber: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- StandardCost: double (nullable = true)
 |-- ListPrice: double (nullable = true)
 |-- Size: string (nullable = true)
 |-- Weight: double (nullable = true)
 |-- ProductCategoryID: integer (nullable = true)
 |-- ProductModelID: integer (nullable = true)
 |-- SellStartDate: string (nullable = true)
 |-- SellEndDate: string (nullable = true)
 |-- modifiedDate: string (nullable = true)
 |-- SellDuration: integer (nullable = true)



#### Fill Null Values

In [0]:
df_prod = df_prod.fillna('N/A', subset=['Color', 'Size'])\
                .fillna(0, subset=['Weight'])
df_prod.limit(5).display()

ProductID,Name,ProductNumber,Color,StandardCost,ListPrice,Size,Weight,ProductCategoryID,ProductModelID,SellStartDate,SellEndDate,modifiedDate,SellDuration
680,"HL Road Frame - Black, 58",FR-R92B-58,Black,1059.31,1431.5,58,1016.04,18,6,01/06/2002,08/04/2025,11/03/2008,8347
706,"HL Road Frame - Red, 58",FR-R92R-58,Red,1059.31,1431.5,58,1016.04,18,6,01/06/2002,08/04/2025,11/03/2008,8347
707,"Sport-100 Helmet, Red",HL-U509-R,Red,13.0863,34.99,N/A,0.0,35,33,01/07/2005,08/04/2025,11/03/2008,7221
708,"Sport-100 Helmet, Black",HL-U509,Black,13.0863,34.99,N/A,0.0,35,33,01/07/2005,08/04/2025,11/03/2008,7221
709,"Mountain Bike Socks, M",SO-B909-M,White,3.3963,9.5,M,0.0,27,18,01/07/2005,30/06/2006,11/03/2008,364


#### Check Null Values

In [0]:
df_prod.select([
    sum(col(c).isNull().cast('int')).alias(c) for c in df_prod.columns
]).limit(5).display()

ProductID,Name,ProductNumber,Color,StandardCost,ListPrice,Size,Weight,ProductCategoryID,ProductModelID,SellStartDate,SellEndDate,modifiedDate,SellDuration
0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Write File to Silver Container

In [0]:
df_prod.write.format('delta').mode('overwrite').option('path', 'abfss://silver@dlpracticenajeeb.dfs.core.windows.net/Product').save()